In [1]:
import random
import pandas as pd
import yaml
import joblib
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from modeling import WithDropout
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt

team_rb_stats = pd.read_csv("data/team_rushers.csv", index_col=0)
team_qb_stats = pd.read_csv("data/team_qb_stats.csv", index_col=0)
team_receiver_stats = pd.read_csv("data/team_receiver_stats.csv", index_col=0)
team_stats = pd.read_csv("data/agg/team_stats.csv", index_col=0).rename(
    {"recent_team": "team_name"}, axis=1
)
opp_stats = pd.read_csv("data/agg/opp_stats.csv", index_col=0).rename(
    {"opponent_team": "team_name"}, axis=1
)

In [2]:
players = pd.read_parquet("data/agg/player_weekly_agg.parquet").fillna(0)

In [3]:
players.loc[(players.player_id == "00-0036322") & (players.season == 2024)].iloc[-3]
# will ship '00-0039746'

season                                                                           2024
team_name                                                                         MIN
week                                                                             11.0
game_type                                                                         REG
depth_team                                                                        1.0
last_name                                                                   Jefferson
first_name                                                                     Justin
football_name                                                                  Justin
formation                                                                     Offense
gsis_id                                                                    00-0036322
jersey_number                                                                    18.0
position                                              

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"
model_path = "models/run_yards_gained.pt"
run_yards_model = WithDropout(n_in=22, n_hidden=1000, n_out=130).to(device)
run_yards_model.load_state_dict(torch.load(model_path, weights_only=True))

air_yards_path = "models/air_yards.pt"
air_yards_model = WithDropout(n_in=28, n_out=119)
air_yards_model.load_state_dict(torch.load(air_yards_path, weights_only=True))


yac_model_path = "models/yac.pt"
yac_model = WithDropout(n_in=29, n_out=125)
yac_model.load_state_dict(torch.load(yac_model_path, weights_only=True))

<All keys matched successfully>

In [5]:
stat_cols = [
    "completions",
    "attempts",
    "passing_yards",
    "passing_tds",
    "interceptions",
    "sacks",
    "sack_yards",
    "sack_fumbles",
    "sack_fumbles_lost",
    "passing_air_yards",
    "passing_yards_after_catch",
    "passing_first_downs",
    "passing_epa",
    "passing_2pt_conversions",
    "pacr",
    "dakota",
    "carries",
    "rushing_yards",
    "rushing_tds",
    "rushing_fumbles",
    "rushing_fumbles_lost",
    "rushing_first_downs",
    "rushing_epa",
    "rushing_2pt_conversions",
    "receptions",
    "targets",
    "receiving_yards",
    "receiving_tds",
    "receiving_fumbles",
    "receiving_fumbles_lost",
    "receiving_air_yards",
    "receiving_yards_after_catch",
    "receiving_first_downs",
    "receiving_epa",
    "receiving_2pt_conversions",
    "racr",
    "target_share",
    "air_yards_share",
    "wopr",
    "special_teams_tds",
    "fantasy_points",
    "fantasy_points_ppr",
]

In [6]:
class Player:
    def __init__(self, d):
        self.name = d["full_name"]
        self.id = d["gsis_id"]
        self.depth_team = int(d["dense_depth"])
        self.stats = {x: 0 for x in stat_cols}
        self.stats["air_yards"] = 0
        self.stats["yac"] = 0
        self.features = d.to_dict()

    def reset_stats(self):
        self.stats = {stat_name: 0 for stat_name in self.stats}

    def show_headshot(self):
        url = self.headshot_url
        response = requests.get(url)
        response.raise_for_status()  # Ensure the request was successful
        img = Image.open(BytesIO(response.content))
        plt.imshow(img)
        plt.axis("off")  # Turn off axis for cleaner display
        plt.show()

    def __getattr__(self, name):
        # Redirect attribute access to the stats dictionary
        if name in self.stats:
            return self.stats[name]
        elif name in self.features:
            return self.features[name]
        raise AttributeError(
            f"'{self.__class__.__name__}' object has no attribute '{name}'"
        )

    def __setattr__(self, name, value):
        if name in {
            "stats",
            "features",
            "name",
            "id",
            "depth_team",
        }:  # Handle direct attributes
            super().__setattr__(name, value)
        elif name in self.stats:  # Redirect updates to stats dictionary
            self.stats[name] = value
        elif name in self.features:  # Redirect updates to features dictionary
            self.features[name] = value
        else:
            raise AttributeError(f"Cannot set unknown attribute '{name}'")

    def stats_to_dict(self):
        out = {}
        out["name"] = self.name
        out["id"] = self.id
        out.update(self.stats)

        return out

In [7]:
def fetch_row_or_latest(df, team, season, week):
    df = df.loc[(df.team_name == team) & (df.season == season)]
    row = df.loc[(df.week == min(df.week.max(), week))].to_dict(orient="records")[0]
    return row


class QB(Player):
    def __init__(self, d):  # noqa: F811
        super().__init__(d)
        self.name = d["full_name"]
        self.features = d.to_dict()
        self.position = "QB"

    def __repr__(self):
        return f"QB:{self.name} has {self.completions} completions for {self.passing_yards} yards"


class RB(Player):
    def __init__(self, d):
        super().__init__(d)
        self.name = d["full_name"]
        self.id = d["gsis_id"]
        self.position = "RB"
        self.features = d.to_dict()

    def __repr__(self):
        return (
            f"RB:{self.name} has {self.carries} carries for {self.rushing_yards} yards"
        )


class WR(Player):
    def __init__(self, d):
        super().__init__(d)
        self.name = d["full_name"]
        self.id = d["gsis_id"]
        self.position = "WR"

    def __repr__(self):
        return f"WR:{self.name} has {self.receptions} receptions for {self.receiving_yards} yards"


class TE(Player):
    def __init__(self, d):
        super().__init__(d)
        self.name = d["full_name"]
        self.id = d["gsis_id"]
        self.position = "TE"

    def __repr__(self):
        return f"TE:{self.name} has {self.receptions} receptions for {self.receiving_yards} yards"


class K(Player):
    def __init__(self, d):
        super().__init__(d)
        self.name = d["full_name"]
        self.id = d["gsis_id"]
        self.position = "TE"

    def __repr__(self):
        return f"TE:{self.name} has {self.receptions} receptions for {self.receiving_yards} yards"


class Team:
    def __init__(self, name: str, season: int, week: int, use_current_injuries=False):
        self.name = name
        self.score = 0
        self.plays = 0
        self.features = {"last_rusher_drive": -1, "last_rusher_team": -1}
        self.team_stats = fetch_row_or_latest(team_stats, self.name, season, week)
        self.opp_stats = fetch_row_or_latest(opp_stats, self.name, season, week)
        self.roster = players.loc[
            (players.team_name == name) & (players.season == season)
        ]
        self.roster = self.roster.loc[
            (self.roster.week == min(self.roster.week.max(), week))
            & (self.roster.formation == "Offense")
            & (self.roster.position.isin(["QB", "WR", "TE", "RB", "K"]))
        ].sort_values(by="dense_depth")

        self.QBs = self.get_players_by_position("QB")
        self.RBs = self.get_players_by_position("RB")
        self.WRs = self.get_players_by_position("WR")
        self.TEs = self.get_players_by_position("TE")
        self.players = self.QBs + self.RBs + self.WRs + self.TEs
        self.rb_stats = fetch_row_or_latest(team_rb_stats, self.name, season, week)

        self.team_receiver_stats = fetch_row_or_latest(
            team_receiver_stats, self.name, season, week
        )

    def get_players_by_position(self, position: str):
        """Filter players by position and create player objects."""
        with pd.option_context("future.no_silent_downcasting", True):
            position_data = self.roster[(self.roster["position"] == position)].fillna(0)
        # Create player objects based on position
        players = []
        for _, player_data in position_data.iterrows():
            if position == "WR":
                players.append(WR(player_data))
            elif position == "RB":
                players.append(RB(player_data))
            elif position == "QB":
                players.append(QB(player_data))
            elif position == "TE":
                players.append(TE(player_data))
        return players

    def get_depth_pos(self, pos: str, depth: int):
        """input a position and team depth, to get the player
        used to go from ML output -> player object"""
        while depth >= 0:
            if pos == "WR":
                for player in self.WRs:
                    if player.depth_team == depth:
                        return player
            if pos == "RB":
                for player in self.RBs:
                    if player.depth_team == depth:
                        return player
            if pos == "TE":
                for player in self.TEs:
                    if player.depth_team == depth:
                        return player
            if pos == "QB":
                for player in self.QBs:
                    if player.depth_team == depth:
                        return player
            depth -= 1
        print(pos, depth, self.name)
        raise ValueError("You want a player that does not exist")

    def game_results(self, df=False):
        r = [
            {"team_name": self.name, "position": x.position, "id": x.id}
            | x.stats_to_dict()
            for x in self.players
        ]
        if df:
            return pd.json_normalize(r)
        return r

    def reset_stats(self):
        [x.reset_stats() for x in self.players]
        self.score = 0
        self.plays = 0

    def __repr__(self):
        return f"{self.name} has {self.score} points"

In [8]:
vik = Team("MIN", 2024, 13)
det = Team("DET", 2024, 13)

In [30]:
from abc import ABC, abstractmethod


class Play(ABC):
    def __init__(self, game):
        self.game = game
        self.clock_cols = game.config["clock_cols"]
        self.clock_model = joblib.load("models/clock_model.joblib")
        self.play_context = {}
        self.play_data = {'incomplete_pass':0,
                          'out_of_bounds':0,
                      'player':None,
                      'timeout': 0,
                      'sp': 0}

    @abstractmethod
    def execute_play(self, team):
        return
    
    def log_play(self, play_type, yds, verbose=False):
        """Logs the context of the game state at each play."""
        play_data = {
            "play_type": play_type,
            "yards_gained": yds,
            "player": self.play_data['player'],
        }
        play_data.update(self.game.game_context)
        if verbose:
            print(
                f'{self.game.possession.name} {play_type} for {yds} yards, {self.game.pbp[-1]['yardline_100']} yd line,'
                + f' {self.game.pbp[-1]['ydstogo']} yds to go on {self.game.pbp[-1]['down']} down.'
                + f' {self.game.pbp[-1]['quarter_seconds_remaining'] // 60}:{self.game.pbp[-1]['quarter_seconds_remaining']  % 60} left'
            )
        self.game.pbp.append(play_data)
        return play_data
    
    def sample_clock(self, play_type):
        #this is the time from the previous play, until the next play starts.
        raw_features = self.collect_features(
            {"next_play": self.game.int_from_play[play_type],
            'play_type_enc': self.game.int_from_play[play_type]},
             self.game.pbp[-1],
            self.play_data
        )
        # print(self.pbp[-1])
        features = np.array([[raw_features[key] for key in self.clock_cols]])
        return self.clock_model.predict(features).item()
        #return random.randint(15, 40)
    
    def orchestrate(self, team):
        self.execute_play(team)

    def collect_features(self, *argv):
        features = {}
        features.update(self.game.game_context)
        for arg in argv:
            features.update(arg)
        return features


class RunPlay(Play):
    def __init__(self, game):
        super().__init__(game)
        self.choose_rusher_model = joblib.load("models/choose_rusher.joblib")
        self.choose_rusher_cols = game.config["choose_rusher_cols"]
        self.rusher_idx_to_pos = game.config["rusher_idx_to_pos"]
        self.rush_yard_cols = game.config["rush_yard_cols"]

    def choose_rusher(self, team):
        raw_features = self.collect_features(
            team.rb_stats,
            team.features,
            self.game.defending.opp_stats,
        )
        features = [raw_features[key] for key in self.choose_rusher_cols]
        preds = self.choose_rusher_model.predict_proba([features])
        rusher_idx = np.random.choice(len(preds[0]), p=preds[0])
        pos, depth = self.rusher_idx_to_pos[rusher_idx].split("_")
        player = team.get_depth_pos(pos, int(depth))
        team.features["last_rusher_team"] = rusher_idx
        team.features["last_rusher_drive"] = rusher_idx
        return player

    def sample_run_yards(self, model, team, player):
        raw_features = self.collect_features(
            player.features,
            team.team_stats,
            team.opponent.opp_stats,
        )
        x = [raw_features[key] for key in self.rush_yard_cols]
        x = torch.tensor(x).to(device)
        with torch.no_grad():
            preds = model(x.reshape(1, -1))[0]
            preds = torch.softmax(preds, 0)
        sample = (torch.multinomial(preds, 1)).item() - 30
        return min((sample, raw_features["yardline_100"]))

    def execute_play(self, team):
        player = self.choose_rusher(team)
        yds = self.sample_run_yards(run_yards_model, team, player)
        player.carries += 1
        player.rushing_yards += yds
        self.game.player = player.name
        return yds


class PassPlay(Play):
    def __init__(self, game):
        super().__init__(game)
        self.choose_receiver = joblib.load("models/choose_receiver.joblib")
        self.choose_receiver_cols = game.config["choose_receiver_cols"]
        self.air_yards_cols = game.config["air_yards_cols"]
        self.receiver_idx_to_pos = game.config["receiver_idx_to_pos"]
        self.complete_pass_cols = game.config["complete_pass_cols"]
        self.complete_pass_model = joblib.load("models/complete_pass.joblib")

    def sample_air_and_yac(self, air_model, yac_model, team, player):
        raw_features = self.collect_features(
            player.features,
            team.team_stats,
            team.opponent.opp_stats,
        )
        x = [raw_features[key] for key in self.air_yards_cols]
        x = torch.tensor(x)
        with torch.no_grad():
            preds = air_model(x.reshape(1, -1))[0]
            preds = torch.softmax(preds, 0)
        air_yards = (torch.multinomial(preds, 1)).item() - 20
        if air_yards >= self.game.ball_position:  # touchdown at catch
            return self.game.ball_position, 0

        x = torch.cat((x, torch.tensor([air_yards])))
        with torch.no_grad():
            preds = yac_model(x.reshape(1, -1))[0]
            preds = torch.softmax(preds, 0)
        yac = (torch.multinomial(preds, 1)).item() - 25
        yac = min(yac, (self.game.ball_position - air_yards))
        self.play_context.update({'air_yards':air_yards,
                          'yac': yac})
        return air_yards, yac

    def sample_completion(self, qb, receiver, team, air_yards):
        raw_features = self.collect_features(
            receiver.features,
            team.team_stats,
            team.opponent.opp_stats,
        )
        raw_features["air_yards"] = air_yards
        qb_features = {(key + "_qb"): value for key, value in qb.features.items()}
        raw_features.update(qb_features)
        features = [raw_features[key] for key in self.complete_pass_cols]
        preds = self.complete_pass_model.predict_proba([features])
        receiver = np.random.choice(len(preds[0]), p=preds[0])
        return np.random.choice(len(preds[0]), p=preds[0])
    
    def get_receiver(self, team):
        raw_features = self.collect_features(
            team.team_receiver_stats,
            team.features,
        )
        features = [raw_features[key] for key in self.choose_receiver_cols]
        preds = self.choose_receiver.predict_proba([features])
        receiver = np.random.choice(len(preds[0]), p=preds[0])
        pos, depth = self.receiver_idx_to_pos[receiver].split("_")
        receiver = team.get_depth_pos(pos, int(depth))
        return receiver

    def execute_play(self, team):
        passer = team.QBs[0]
        receiver = self.get_receiver(team)
        passer.attempts += 1
        receiver.targets += 1
        air_yards, yac = self.sample_air_and_yac(
            air_yards_model, yac_model, team, receiver
        )

        if self.sample_completion(passer, receiver, team, air_yards):
            passer.completions += 1
            receiver.receptions += 1
            yds = air_yards + yac
            receiver.air_yards += air_yards
            receiver.yac += yac
            receiver.receiving_yards += yds
            passer.passing_yards += yds
        else:
            yds = 0
        self.play_data['player'] = receiver.name
        return yds


class FieldGoal(Play):
    def __init__(self, game):
        super().__init__(game)

    def execute_play(self, team):
        result = random.randint(0, 100)
        if result > (10 + (1.7 * self.game.ball_position)):
            team.score += 3
            self.game.switch_poss()
            self.game.ball_position = 65
            # print(f'{team.name} scored a FG')
        else:
            # print(f'{team.name} missed FG')
            self.game.switch_poss()
        self.game.player = None
        return 0


class Punt(Play):
    def __init__(self, game):
        super().__init__(game)

    def execute_play(self, team):
        self.game.switch_poss()
        self.game.ball_position += random.randint(35, 60)
        if self.game.ball_position >= 100:
            self.game.ball_position = 20
        self.game.player = None
        return 0


class Kneel(Play):
    def __init__(self, game):
        super().__init__(game)

    def execute_play(self, team):
        # Implementation of qb kneel play
        # print("QB kneel executed.")
        return -1


class Spike(Play):
    def __init__(self, game):
        super().__init__(game)

    def execute_play(self, team):
        # Implementation of qb spike play
        # print("QB spike executed.")
        return -1


class PosTimeout(Play):
    def __init__(self, game):
        super().__init__(game)

    def execute_play(self, team):
        self.game.possession.timeouts -= 1
        #print(f"TIMEOUT! {self.game.possession.timeouts} remaining")
        return 0


class DefTimeout(Play):
    def __init__(self, game):
        super().__init__(game)

    def execute_play(self, team):
        self.game.defending.timeouts -= 1
        #print(f"def TIMEOUT!, {self.game.defending.timeouts} remaining")
        return 0



In [21]:
class GameState:
    def __init__(self, home, away, config, **kwargs):
        self.config = config
        self.home = home
        self.away = away
        home.opponent = self.away
        away.opponent = self.home
        home.spread_line = kwargs.get("spread_line", -3)
        away.spread_line = -1 * self.home.spread_line
        self.total_line = kwargs.get("total_line", 42)
        self.run_or_pass = joblib.load("models/run_or_pass.joblib")
        self.run_or_pass_cols = config["run_or_pass_cols"]
        self.play_encoding = config["play_encoding"]
        self.int_from_play = {v:k for k,v in self.play_encoding.items()}
        self.wind = kwargs.get("wind", random.randint(0, 10))
        self.temp = kwargs.get("temp", random.randint(40, 90))
        self.play_functions = {
            "field_goal": FieldGoal(self),
            "no_play": RunPlay(self),
            "pass": PassPlay(self),
            "punt": Punt(self),
            "qb_kneel": Kneel(self),
            "qb_spike": Spike(self),
            "run": RunPlay(self),
            "pos_timeout": PosTimeout(self),
            "def_timeout": DefTimeout(self),
        }
        self.reset_game()
        self.game_context = self.get_game_state()   

    def reset_game(self):
        self.quarter = 1
        self.possession = self.home
        self.defending = self.away
        self.down = 1
        self.ydstogo = 10
        self.ball_position = 65  # Yardline (0-100), 0 is score, 100 is safety
        self.clock = 900  # Seconds in the current quarter (15 mins = 900 seconds)
        self.drive = 0
        self.pbp = []
        self.player = None
        self.home.timeouts = 3
        self.away.timeouts = 3
        self.home.reset_stats()
        self.away.reset_stats()

    def switch_poss(self):
        self.possession.features["last_rusher_drive"] = -1
        self.possession = self.away if self.possession == self.home else self.home
        self.defending = self.possession.opponent
        self.ball_position = 100 - min(self.ball_position, 99)
        self.down = 1
        self.ydstogo = min(10, self.ball_position)
        self.drive += 1
        return

    def kickoff(self):
        self.switch_poss()
        self.ball_position = 65
        self.log_play('kickoff',0)
        pass

    def start_game(self):
        self.lost_kickoff = random.choice((self.home, self.away))
        self.possession = self.lost_kickoff
        self.kickoff()
        self.game_context = self.get_game_state()
        # print(f"{self.possession.name} has won the kickoff")
        self.log_play("kickoff", 0)

    def collect_features(self, *argv):
        features = {}
        features.update(self.game_context)
        for arg in argv:
            features.update(arg)
        return features

    def get_game_state(self):
        """Logs the context of the game state at each play."""
        play_data = {
            "possession": self.possession.name,
            "quarter": self.quarter,
            "down": self.down,
            "ydstogo": self.ydstogo,
            "goal_to_go": int(self.ball_position < 10),
            "yardline_100": self.ball_position,
            "total_home_score": self.home.score,
            "total_away_score": self.away.score,
            "posteam_score": self.possession.score,
            "defteam_score": self.defending.score,
            "score_differential": (self.possession.score - self.defending.score),
            "wind": self.wind,
            "temp": self.temp,
            "quarter_seconds_remaining": self.clock,
            "half_seconds_remaining": self.clock + (900 * (self.quarter % 2)),
            "game_seconds_remaining": self.clock + (900 * (4 - self.quarter)),
            "drive": self.drive,
            "spread_line": self.possession.spread_line,
            "total_line": self.total_line,
            "posteam_timeouts_remaining": self.possession.timeouts,
            "defteam_timeouts_remaining": self.defending.timeouts,
        }
        return play_data

    def play(self, team):
        self.game_context = self.get_game_state()
        team.plays += 1
        raw_features = self.collect_features(
            team.team_stats,
            self.defending.opp_stats,
        )
        features = [raw_features[key] for key in self.run_or_pass_cols]
        preds = self.run_or_pass.predict_proba([features])
        if self.possession.timeouts == 0:
            preds[:, 7] = 0
            preds /= preds.sum()
        if self.defending.timeouts == 0:
            preds[:, 8] = 0
            preds /= preds.sum()
        play_type_int = np.random.choice(len(preds[0]), p=preds[0])
        play_type = self.play_encoding.get(play_type_int, 1)
        self.clock -= self.play_functions[play_type].sample_clock(play_type)
        yds = self.play_functions[play_type].execute_play(team)
        self.log_play(play_type, yds)
        if play_type not in ["field_goal", "punt", "pos_timeout", "def_timeout"]:
            self.ydstogo -= yds
            self.ball_position -= yds
            self.td_check(team)
            self.check_downs(team)

    def log_play(self, play_type, yds, verbose=False):
        """Logs the context of the game state at each play."""
        play_data = {
            "play_type": play_type,
            "yards_gained": yds,
            "player": self.player,
        }
        play_data.update(self.game_context)
        if verbose:
            print(
                f'{self.possession.name} {play_type} for {yds} yards, {self.pbp[-1]['yardline_100']} yd line,'
                + f' {self.pbp[-1]['ydstogo']} yds to go on {self.pbp[-1]['down']} down.'
                + f' {self.pbp[-1]['quarter_seconds_remaining'] // 60}:{self.pbp[-1]['quarter_seconds_remaining']  % 60} left'
            )
        self.pbp.append(play_data)
        return play_data

    def td_check(self, team):
        if self.ball_position <= 0:
            team.score += 7
            self.pbp[-1]
            self.kickoff()
            # print(f'{team.name} scored a TD')
        return

    def check_downs(self, team):
        if self.ydstogo <= 0:
            self.ydstogo = 10
            self.down = 1
        elif self.down == 4:
            self.switch_poss()
        else:
            self.down += 1

    def play_quarter(self):
        self.clock = 900
        if self.quarter == 3:
            self.kickoff()
            self.possession = self.lost_kickoff
        if self.quarter == 3:
            self.home.timeouts = 3
            self.away.timeouts = 3
        while self.clock > 0:
            self.play(self.possession)
        self.quarter += 1
        # print(f"{self.home.name}:{self.home.score}")
        # print(f"{self.away.name}:{self.away.score}")

    def play_game(self):
        self.reset_game()
        self.start_game()
        while self.quarter <= 4:
            self.play_quarter()
            # print(self.quarter)
        if self.home.score > self.away.score:
            print(f"{self.home.name} has won {self.home.score} - {self.away.score}")
        else:
            print(f"{self.away.name} has won {self.away.score} - {self.home.score}")

    def game_results(self, df=False):
        res1 = self.home.game_results(df=False)
        res2 = self.away.game_results(df=False)
        res = res1 + res2
        if df:
            return pd.DataFrame(res)
        return res

In [22]:
p = PassPlay(game)
p.execute_play(vik)

11

In [23]:
p.log_play('pass',10,verbose=True)

LV pass for 10 yards, 52 yd line, 5 yds to go on 2 down. 0.0:6.19691726565361 left


{'play_type': 'pass',
 'yards_gained': 10,
 'player': 'Dallas Goedert',
 'possession': 'LV',
 'quarter': 4,
 'down': 2,
 'ydstogo': 5,
 'goal_to_go': 0,
 'yardline_100': 52,
 'total_home_score': 30,
 'total_away_score': 28,
 'posteam_score': 28,
 'defteam_score': 30,
 'score_differential': -2,
 'wind': 5,
 'temp': 90,
 'quarter_seconds_remaining': 6.19691726565361,
 'half_seconds_remaining': 6.19691726565361,
 'game_seconds_remaining': 6.19691726565361,
 'drive': 22,
 'spread_line': 12.5,
 'total_line': 42,
 'posteam_timeouts_remaining': 0,
 'defteam_timeouts_remaining': 0}

In [36]:
with open("models/feature_config.yaml", "r") as file:
    config = yaml.safe_load(file)

vik = Team("PHI", 2024, 13)
det = Team("LV", 2024, 13)
game = GameState(vik, det, config, **{"wind": 5, "temp": 90, "spread_line": -12.5})
game.play_game()
game.game_results(df=True).groupby(["team_name"]).sum()[
    ["receiving_yards", "rushing_yards"]
]


LV has won 17 - 10


,receiving_yards,rushing_yards
team_name,,
LV,210,79
PHI,173,98


In [ ]:
a = []
for i in range(3000):  # 2.2 with gpu
    a.append(game.sample_air_and_yac(air_yards_model, yac_model, det.WRs[0]))

AttributeError: 'GameState' object has no attribute 'sample_air_and_yac'

In [ ]:
air = [x[0] for x in a]
plt.hist(air, bins=max(air) - min(air))

ValueError: max() iterable argument is empty

In [ ]:
pd.DataFrame(air).value_counts()

TypeError: Index(...) must be called with a collection of some kind, 'air_yards' was passed

In [ ]:
max(air) - min(air)

64

In [ ]:
def run_sim(team1, team2, num_sims, config=config):
    results = []
    scores = []
    vik = Team("CAR", 2024, 13)
    det = Team("TB", 2024, 13)
    game = GameState(vik, det, config, **{"wind": 5, "temp": 60, "spread_line": 6.5})
    for i in range(num_sims):
        game.start_game()
        game.play_game()
        scores.append((game.home.score, game.away.score))
        res = vik.game_results() + det.game_results()
        for result in res:
            result["game_number"] = i
        results += res
    return results, scores


results, scores = run_sim(
    "KC", "LV", 5
)  # 28.8 -> 22.8 by pulling game initialization out of loop

run play initialized
pass play initialized
run play initialized
CAR has won 38 - 28
TB has won 24 - 17
CAR has won 38 - 20
TB has won 21 - 13
TB has won 27 - 24


In [ ]:
sum([x[0] / 50 for x in scores])

14.620000000000001

In [ ]:
(1909 + 2191)

4100

In [ ]:
df = pd.json_normalize(results)
df.groupby(["name", "id"])[
    [
        "receiving_yards",
        "targets",
        "receptions",
        "air_yards",
        "completions",
        "attempts",
        "rushing_yards",
        "carries",
    ]
].agg(["median", "mean"]).sort_values(by=[("receiving_yards", "mean")], ascending=False)

receiving_yards         targets         \
                                          median    mean  median   mean   
name                  id                                                  
Jakobi Meyers         00-0034960            99.0  104.44    14.0  14.01   
Brock Bowers          00-0039338            88.0   92.71    11.5  11.30   
Travis Kelce          00-0030506            69.5   72.33     9.0   9.29   
Tre Tucker            00-0038563            51.0   55.58     7.0   6.97   
DeAndre Hopkins       00-0030564            44.0   47.42     6.0   5.54   
Xavier Worthy         00-0039894            35.0   43.13     5.0   5.64   
Noah Gray             00-0036637            30.0   37.59     5.0   5.35   
D.J. Turner           00-0036527            30.5   33.21     4.0   4.54   
Justin Watson         00-0034386            15.0   24.44     3.0   3.01   
Kareem Hunt           00-0033923            17.0   23.37     4.0   4.18   
JuJu Smith-Schuster   00-0033857             8.5   16.68     2.0   2.53   
Carson Steele         00-0039325             5.5   11.58     1.0   1.61   
Harrison Bryant       00-0036232             3.0    9.25     1.0   1.51   
Alexander Mattison    00-0034972             4.5    8.23     1.0   1.26   
Mecole Hardman        00-0035140             2.0    7.31     1.0   1.10   
Ramel Keyton          00-0039543             0.0    5.71     1.0   0.80   
Peyton Hendershot     00-0037569             0.0    4.29     0.0   0.49   
Zamir White           00-0038040             0.0    1.20     0.0   0.30   
Justin Shorter        00-0038590             0.0    0.85     0.0   0.07   
Ameer Abdullah        00-0032104             0.0    0.00     0.0   0.00   
Gardner Minshew       00-0035289             0.0    0.00     0.0   0.00   
Dylan Laube           00-0039407             0.0    0.00     0.0   0.00   
Patrick Mahomes       00-0033873             0.0    0.00     0.0   0.00   
Samaje Perine         00-0033526             0.0    0.00     0.0   0.00   
Desmond Ridder        00-0038122             0.0    0.00     0.0   0.00   
Clyde Edwards-Helaire 00-0036360             0.0    0.00     0.0   0.00   
Carson Wentz          00-0032950             0.0    0.00     0.0   0.00   

                                 receptions       air_yards         \
                                     median  mean    median   mean   
name                  id                                             
Jakobi Meyers         00-0034960        9.0  8.53      62.0  64.20   
Brock Bowers          00-0039338        8.0  7.86      46.0  51.65   
Travis Kelce          00-0030506        6.0  5.77      37.5  40.24   
Tre Tucker            00-0038563        4.0  4.18      30.0  33.64   
DeAndre Hopkins       00-0030564        4.0  3.66      22.5  26.65   
Xavier Worthy         00-0039894        3.0  3.49      20.0  25.84   
Noah Gray             00-0036637        3.0  3.60      12.5  17.52   
D.J. Turner           00-0036527        3.0  3.24      14.5  16.65   
Justin Watson         00-0034386        2.0  2.17       7.0  14.18   
Kareem Hunt           00-0033923        3.0  3.08       2.0   4.98   
JuJu Smith-Schuster   00-0033857        1.0  1.70       2.0   7.11   
Carson Steele         00-0039325        1.0  1.17       0.0   4.41   
Harrison Bryant       00-0036232        1.0  0.95       0.0   2.63   
Alexander Mattison    00-0034972        1.0  0.95       0.0   3.35   
Mecole Hardman        00-0035140        1.0  0.79       0.0   2.31   
Ramel Keyton          00-0039543        0.0  0.61       0.0   3.03   
Peyton Hendershot     00-0037569        0.0  0.37       0.0   2.43   
Zamir White           00-0038040        0.0  0.17       0.0   0.27   
Justin Shorter        00-0038590        0.0  0.05       0.0   0.54   
Ameer Abdullah        00-0032104        0.0  0.00       0.0   0.00   
Gardner Minshew       00-0035289        0.0  0.00       0.0   0.00   
Dylan Laube           00-0039407        0.0  0.00       0.0   0.00   
Patrick Mahomes       00-0033873

In [ ]:
df.groupby(["team_name", "game_number"])[
    [
        "receiving_yards",
        "targets",
        "receptions",
        "air_yards",
        "completions",
        "attempts",
        "rushing_yards",
        "carries",
    ]
].sum().groupby(["team_name"]).mean().sort_values(
    by=["receiving_yards"], ascending=False
)

,receiving_yards,targets,receptions,air_yards,completions,attempts,rushing_yards,carries
team_name,,,,,,,,
LV,311.18,40.76,26.54,175.96,26.54,40.76,131.48,27.77
KC,288.14,38.74,25.80,145.67,25.80,38.74,115.59,25.45


In [ ]:
sched = pd.read_csv("week_13.csv", index_col=0)
sched.iloc[11]

game_id           2024_13_TB_CAR
season                      2024
week                          13
home_team                    CAR
away_team                     TB
home_moneyline             205.0
spread_line                 -6.5
total_line                  46.5
temp                        50.0
wind                         8.0
Name: 6896, dtype: object

In [ ]:
def run_sim(team1, team2, num_sims, sched, config=config):
    results = []
    scores = []
    for i in range(num_sims):
        vik = Team(sched["home_team"], sched["season"], sched["week"])
        det = Team(sched["away_team"], sched["season"], sched["week"])
        game = GameState(vik, det, config, **sched)
        game.start_game()
        game.play_game()
        scores.append((game.home.score, game.away.score))
        res = vik.game_results() + det.game_results()
        for result in res:
            result["game_number"] = i
        results += res
    return results, scores


# results, scores = run_sim('KC','LV',100)

In [ ]:
results = []
test = True
if test:
    for t in [11]:
        # for t in range(len(sched)):
        game1 = sched.iloc[t].to_dict()
        try:
            results, scores = run_sim("KC", "LV", 100, game1)
        except:
            print("oops")

CAR has won 13 - 7
CAR has won 31 - 7
CAR has won 30 - 6
CAR has won 26 - 17
TB has won 28 - 24
CAR has won 34 - 27
TB has won 24 - 14
TB has won 27 - 20
TB has won 28 - 24
CAR has won 24 - 20
CAR has won 17 - 13
CAR has won 21 - 13
CAR has won 40 - 14
CAR has won 24 - 14
TB has won 16 - 16
TB has won 20 - 19
TB has won 23 - 21
CAR has won 26 - 17
TB has won 21 - 17
TB has won 26 - 17
TB has won 17 - 17
CAR has won 13 - 10
CAR has won 24 - 9
CAR has won 45 - 17
CAR has won 44 - 17
TB has won 31 - 23
CAR has won 31 - 21
TB has won 20 - 7
TB has won 17 - 17
CAR has won 23 - 17
TB has won 27 - 6
CAR has won 28 - 27
TB has won 21 - 16
TB has won 24 - 6
TB has won 24 - 14
CAR has won 41 - 16
CAR has won 23 - 17
CAR has won 41 - 38
TB has won 31 - 14
TB has won 20 - 17
TB has won 34 - 14
CAR has won 20 - 10
TB has won 10 - 10
TB has won 31 - 10
CAR has won 34 - 17
TB has won 24 - 23
TB has won 21 - 7
TB has won 26 - 26
CAR has won 41 - 16
CAR has won 20 - 9
TB has won 24 - 24
CAR has won 24 

In [ ]:
sum([x[0] for x in scores])

2153

In [ ]:
weekly = pd.read_csv("data/weekly.csv", index_col=0)
pit = weekly.loc[
    (weekly.week == 12)
    & (weekly.season == 2024)
    & ((weekly.recent_team == "PIT") | (weekly.recent_team == "CLE"))
]

In [ ]:
df = pd.json_normalize(results)
check_stats = [
    "receiving_yards",
    "targets",
    "receptions",
    "completions",
    "attempts",
    "rushing_yards",
    "carries",
]

preds = (
    df.groupby(["name", "team_name", "id"])[
        [
            "receiving_yards",
            "targets",
            "receptions",
            "air_yards",
            "completions",
            "attempts",
            "rushing_yards",
            "carries",
        ]
    ]
    .quantile(0.5)
    .sort_values(by=["rushing_yards"], ascending=False)
    .reset_index()
)
preds

KeyError: 'name'

In [ ]:
pp = pd.read_csv(
    "C:/Github/NBA_betting_model/Lines/current/pp_current.csv", index_col=0
)
pp.loc[pp.league_name == "NFL"]["stat"].unique()

array(['Rush+Rec TDs', 'Receiving Yards', 'Receptions', 'Fantasy Score',
       'Rec Targets', 'Longest Reception',
       'Receiving Yards in First 2 Receptions', 'Pass TDs',
       'Pass Completions', 'Completion Percentage', 'Pass Attempts',
       'Pass+Rush Yds', 'Pass Yards', 'Rush Yards', 'INT',
       'Rush Yards in First 5 Attempts', 'Rush Attempts', 'Rush+Rec Yds',
       'Longest Rush', 'FG', 'Pass Yards (Combo)', 'Rush Yards (Combo)',
       'Receiving Yards (Combo)', 'Rush+Rec Yds (Combo)', 'Sacks',
       'Longest FG Made Yds (Combo)', 'Shortest FG Made Yds (Combo)',
       'Kicking Points', 'FG Made (Combo)', 'Punts Inside 20'],
      dtype=object)

In [ ]:
preds2pp = {
    "receiving_yards": "Receiving Yards",
    "receptions": "Receptions",
    "targets": "Rec Targets",
    "completions": "Pass Completions",
    "attempts": "Pass Attempts",
    "passing_yards": "Pass Yards",
    "carries": "Rush Attempts",
}

In [ ]:
df.loc[df.name == "Brock Bowers", "receiving_yards"]

4025     64
4052    175
4079    159
4106    187
4133    121
4160    126
4187     38
4214     55
4241    167
4268    199
4295    179
4322    172
4349    148
4376     81
4403    206
4430    123
4457    167
4484     76
4511    100
4538    101
4565    126
4592    161
4619     55
4646    259
4673    145
4700    220
4727     63
4754     75
4781    146
4808     28
4835    196
4862    208
4889    124
4916     93
4943    112
4970     92
4997     93
5024     90
5051    151
5078    168
5105     86
5132    108
5159    128
5186    123
5213    179
5240    100
5267    189
5294    186
5321     54
5348     47
Name: receiving_yards, dtype: int64

In [ ]:
melted = preds.melt(["name", "team_name", "id"])
melted["stat"] = melted["variable"].apply(lambda x: preds2pp.get(x, x))
comb = melted.merge(
    pp.loc[(pp.league_id == 9) & (pp.alt_line == "standard")],
    left_on=["name", "team_name", "stat"],
    right_on=["player", "team", "stat"],
)
comb["Z"] = (comb["value"] - comb["line"]) / (comb["value"])
comb.loc[comb.Z > -50].sort_values(by="Z")

,name,team_name,id,variable,value,stat,player,team,line,alt_line,opp,league_id,league_name,event_time,pp_player_id,date,scrape_time,prop_id,event_id,Z
70,Brock Wright,DET,00-0036754,receiving_yards,1.0,Receiving Yards,Brock Wright,DET,4.5,standard,CHI,9,NFL,2024-11-28 11:00:00-06:00,215927,2024-11-28 00:00:00-06:00,2024-11-27 19:40:00-06:00,fd12a231-e,a43ef946-7,-3.500000
67,Jalen Tolbert,DAL,00-0037666,receiving_yards,6.0,Receiving Yards,Jalen Tolbert,DAL,21.5,standard,NYG,9,NFL,2024-11-28 15:00:00-06:00,211429,2024-11-28 00:00:00-06:00,2024-11-27 19:40:00-06:00,e196374e-e,00cbc165-f,-2.583333
68,Rico Dowdle,DAL,00-0036139,receiving_yards,4.5,Receiving Yards,Rico Dowdle,DAL,15.5,standard,NYG,9,NFL,2024-11-28 15:00:00-06:00,211732,2024-11-28 00:00:00-06:00,2024-11-27 19:40:00-06:00,c57c0b13-b,00cbc165-f,-2.444444
55,Mike Evans,TB,00-0031408,receiving_yards,18.0,Receiving Yards,Mike Evans,TB,59.5,standard,CAR,9,NFL,2024-12-01 15:00:00-06:00,211278,2024-12-01 00:00:00-06:00,2024-11-27 19:40:00-06:00,42be8414-e,77c97f1c-c,-2.305556
66,Jahmyr Gibbs,DET,00-0039139,receiving_yards,6.5,Receiving Yards,Jahmyr Gibbs,DET,20.5,standard,CHI,9,NFL,2024-11-28 11:00:00-06:00,206307,2024-11-28 00:00:00-06:00,2024-11-27 19:40:00-06:00,46c0b425-e,a43ef946-7,-2.153846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Malik Washington,MIA,00-0039880,receiving_yards,10.5,Receiving Yards,Malik Washington,MIA,5.5,standard,GB,9,NFL,2024-11-28 19:00:00-06:00,216934,2024-11-28 00:00:00-06:00,2024-11-27 19:40:00-06:00,72ddd318-9,73168dea-1,0.476190
43,Josh Jacobs,GB,00-0035700,receiving_yards,30.0,Receiving Yards,Josh Jacobs,GB,15.5,standard,MIA,9,NFL,2024-11-28 19:00:00-06:00,206303,2024-11-28 00:00:00-06:00,2024-11-27 19:40:00-06:00,a64149b4-d,73168dea-1,0.483333
45,Josh Oliver,MIN,00-0035249,receiving_yards,29.0,Receiving Yards,Josh Oliver,MIN,14.5,standard,ARI,9,NFL,2024-12-01 12:00:00-06:00,212257,2024-12-01 00:00:00-06:00,2024-11-27 19:40:00-06:00,711af35e-a,22622b6e-5,0.500000
0,Brock Bowers,LV,00-0039338,receiving_yards,125.0,Receiving Yards,Brock Bowers,LV,61.5,standard,KC,9,NFL,2024-11-29 14:00:00-06:00,210636,2024-11-29 00:00:00-06:00,2024-11-27 19:40:00-06:00,3e740a34-1,278e4fb4-3,0.508000


In [ ]:
comb.groupby(["stat"])[["value", "line"]].mean()

,value,line
stat,,
Pass Attempts,30.055556,31.666667
Pass Completions,19.555556,20.944444
Rec Targets,6.448718,6.179487
Receiving Yards,39.506579,39.947368
Receptions,3.690476,3.952381
Rush Attempts,13.250000,11.000000


In [ ]:
comb = preds.merge(
    pit[["player_id"] + check_stats],
    left_on="id",
    right_on="player_id",
    suffixes=["_pred", "_act"],
).drop("player_id", axis=1)
comb

,name,id,receiving_yards_pred,targets_pred,receptions_pred,air_yards,completions_pred,attempts_pred,rushing_yards_pred,carries_pred,receiving_yards_act,targets_act,receptions_act,completions_act,attempts_act,rushing_yards_act,carries_act
0,George Pickens,00-0037247,104.75,11.50,7.25,63.25,0.00,0.0,0.00,0.00,48.0,7,4,0,0,0.0,0
1,David Njoku,00-0033885,85.25,10.25,9.00,31.50,0.00,0.0,0.00,0.00,9.0,5,1,0,0,0.0,0
2,Jerry Jeudy,00-0036407,81.00,8.00,5.00,49.00,0.00,0.0,0.00,0.00,85.0,6,6,0,0,0.0,0
3,Pat Freiermuth,00-0036894,39.50,4.00,3.25,8.00,0.00,0.0,0.00,0.00,59.0,4,4,0,0,0.0,0
4,Cedric Tillman,00-0038979,38.25,7.25,4.00,27.00,0.00,0.0,0.00,0.00,28.0,4,2,0,0,0.0,0
5,Jaylen Warren,00-0037228,29.00,3.00,3.00,10.00,0.00,0.0,60.00,15.00,19.0,5,3,0,0,45.0,11
6,Jerome Ford,00-0037267,27.00,2.25,2.25,4.00,0.00,0.0,22.25,6.25,8.0,1,1,0,0,19.0,4
7,Najee Harris,00-0036893,26.25,3.00,2.00,2.00,0.00,0.0,102.25,23.00,13.0,2,2,0,0,41.0,16
8,Elijah Moore,00-0036980,23.75,4.25,2.25,17.25,0.00,0.0,0.00,0.00,21.0,5,3,0,0,0.0,0
9,Darnell Washington,00-0038558,20.25,4.00,3.00,14.00,0.00,0.0,0.00,0.00,14.0,3,3,0,0,0.0,0


In [ ]:
comb[
    [
        "air_yards",
        "attempts_act",
        "attempts_pred",
        "carries_act",
        "carries_pred",
        "completions_act",
        "completions_pred",
        "id",
        "name",
        "receiving_yards_act",
        "receiving_yards_pred",
        "receptions_act",
        "receptions_pred",
        "rushing_yards_act",
        "rushing_yards_pred",
        "targets_act",
        "targets_pred",
    ]
]

,air_yards,attempts_act,attempts_pred,carries_act,carries_pred,completions_act,completions_pred,id,name,receiving_yards_act,receiving_yards_pred,receptions_act,receptions_pred,rushing_yards_act,rushing_yards_pred,targets_act,targets_pred
0,63.25,0,0.0,0,0.00,0,0.00,00-0037247,George Pickens,48.0,104.75,4,7.25,0.0,0.00,7,11.50
1,31.50,0,0.0,0,0.00,0,0.00,00-0033885,David Njoku,9.0,85.25,1,9.00,0.0,0.00,5,10.25
2,49.00,0,0.0,0,0.00,0,0.00,00-0036407,Jerry Jeudy,85.0,81.00,6,5.00,0.0,0.00,6,8.00
3,8.00,0,0.0,0,0.00,0,0.00,00-0036894,Pat Freiermuth,59.0,39.50,4,3.25,0.0,0.00,4,4.00
4,27.00,0,0.0,0,0.00,0,0.00,00-0038979,Cedric Tillman,28.0,38.25,2,4.00,0.0,0.00,4,7.25
5,10.00,0,0.0,11,15.00,0,0.00,00-0037228,Jaylen Warren,19.0,29.00,3,3.00,45.0,60.00,5,3.00
6,4.00,0,0.0,4,6.25,0,0.00,00-0037267,Jerome Ford,8.0,27.00,1,2.25,19.0,22.25,1,2.25
7,2.00,0,0.0,16,23.00,0,0.00,00-0036893,Najee Harris,13.0,26.25,2,2.00,41.0,102.25,2,3.00
8,17.25,0,0.0,0,0.00,0,0.00,00-0036980,Elijah Moore,21.0,23.75,3,2.25,0.0,0.00,5,4.25
9,14.00,0,0.0,0,0.00,0,0.00,00-0038558,Darnell Washington,14.0,20.25,3,3.00,0.0,0.00,3,4.00
